# RAG Evaluation with DeepEval

## Introduction
---

[**DeepEval**](https://docs.confident-ai.com/) is a comprehensive framework designed to evaluate the performance and capabilities of large language models (LLMs). It provides a structured and systematic approach to assessing the strengths and weaknesses of these powerful AI systems across a wide range of tasks and domains. By leveraging a diverse set of evaluation metrics, `DeepEval` enables researchers, developers, and users to gain insights into the linguistic, reasoning, and knowledge capabilities of LLMs. This framework aims to foster transparency, reproducibility, and fairness in the evaluation process, ultimately contributing to the responsible development and deployment of these cutting-edge language technologies.

To work with `DeepEval`, we will need to prepare <u>three mandatory components</u>:
1. **Evaluation dataset**: this is a collection of `LLMTestCases` and/or `Goldens` dataset with the expected answer.
2. **Test case(s)**: is a blueprint to unit test LLM outputs. There are two types of test cases in **DeepEval**: `LLMTestCase` and `ConversationalTestCase`.
3. **Metrics**: the standard of measurement, or bring your own metrics for evaluating the performance of an LLM output based your objectives and use cases

<div class="alert alert-block alert-info">
    <b>Note</b>: We have already set up the <b>evaluation dataset</b> during our prerequisite step.
</div>



## Set up

In [1]:
%pip install -qU --quiet -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## RAG Evaluation
---
In general, core components of RAG pipeline involve **retrieval** and **generation** steps, which influenced by hyperparameters. For example, your embedding model choice, search strategy, or number of chunks/nodes to retrieve, LLM temperature, prompt template.

**DeepEval** offers evaluation framework for both retrieval and generation steps separately. This decouple approaches allows the AI developers for easier debugging, and pinpointing the issue, and which components to improve.


<img src='./img/DeepEval-RAG_Eval.png' alt="DeepEval RAG Evaluation" style='width: 500px;'/>

### Get the evaluation dataset
---
Load our prerequisite evaluation dataset, for the detail steps on creating this dataset, please refer to the [prerequisite notebook](../prerequisite-vector-db-and-evaluation-dataset.ipynb)

In [2]:
import pandas as pd
import ast

eval_df = pd.read_csv('../_eval_data/eval_dataframe.csv')
eval_df['source_chunk'] = eval_df.source_chunk.apply(lambda s: list(ast.literal_eval(s)))
eval_df.head(2)

,question,compressed_question,ref_answer,source_sentence,source_chunk,source_document,groundedness_rating,groundedness_reason,relevance_rating,relevance_reason
0,What are the names of the chips that were anno...,Which 2nd-gen chipsets are being utilized by A...,Trainium and Inferentia.,announced second versions of our Trainium and ...,[announced second versions of our Trainium and...,{'source': '_raw_data/AMZN-2023-Shareholder-Le...,5.0,The context provides all the necessary informa...,1.0,The question is not relevant to a business ana...
1,What are the company's priorities in terms of ...,What are key spend & cultural priorities for a...,The company's priorities in terms of spending ...,We will work hard to spend wisely and maintain...,"[the present value of future cash flows, we’ll...",{'source': '_raw_data/AMZN-2021-Shareholder-Le...,5.0,The context provides all the necessary informa...,5.0,The question is very relevant to a business an...


### Connect to existing vector database
---

Connect to our prerequisite vector database, please refer to the [prerequisite notebook](../prerequisite-vector-db-and-evaluation-dataset.ipynb) for setting up vector database.

In [3]:
from langchain_chroma import Chroma
from langchain_aws import BedrockEmbeddings
import boto3


chroma_db_dir = './../vector_db'
chroma_collection_name = 'amazon-shareholder-letters'
boto_session = boto3.session.Session()
titan_model_id = 'amazon.titan-embed-text-v2:0'
titan_embedding_fn = BedrockEmbeddings(
    model_id=titan_model_id,
    region_name=boto_session.region_name
)

vector_store = Chroma(
    collection_name=chroma_collection_name,
    embedding_function=titan_embedding_fn,
    persist_directory=chroma_db_dir,
)

chroma_retriver = vector_store.as_retriever(
    search_kwargs={'k': 3}
)

We can check if the connection is successful by using `vector_store.get()`

In [4]:
len(vector_store.get().get('ids', []))  # here will output the number of docs or chunks within the Chroma DB

322

### Custom LLM Evaluation
---

Before we can use the predefined metrics from `DeepEval`, we need to define our own class. When we define the evaluation **metrics**, we need to pass `model` parameter, otherwise, it will use **OpenAI** as a default model.

Below is the example from `ContextualPrecisionMetric` class:

```{py3}
class ContextualPrecisionMetric(BaseMetric):
    def __init__(
        self,
        threshold: float = 0.5,
        model: Optional[Union[str, DeepEvalBaseLLM]] = None,
        include_reason: bool = True,
        async_mode: bool = True,
        strict_mode: bool = False,
        verbose_mode: bool = False,
    ):
    ...
```

In this notebook, we will use **Llama 3 70B** for RAG response generation and use **Llama 3.1 70B** as evaluation model.

In [5]:
import langchain_aws
from langchain_aws import ChatBedrock

llama3_70b_model_id = 'meta.llama3-70b-instruct-v1:0'
llama3_1_70b_model_id = 'meta.llama3-1-70b-instruct-v1:0'

llama3_70b_langchain = ChatBedrock(
    model_id=llama3_70b_model_id,
    region_name=boto_session.region_name,
    model_kwargs={
        'max_tokens': 2048,
        'temperature': 0.1,
    },
)

llama3_1_70b_langchain = ChatBedrock(
    model_id=llama3_1_70b_model_id,
    region_name=boto_session.region_name,
    model_kwargs={
        'max_tokens': 2048,
        'temperature': 0.1,
    },
)

In [6]:
from deepeval.models import DeepEvalBaseLLM


class BedrockTextGenDeepEval(DeepEvalBaseLLM):
    def __init__(
        self,
        model: langchain_aws.chat_models
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        llm_model = self.load_model()
        return llm_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        llm_model = self.load_model()
        res = await llm_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        llm_model = self.load_model()
        return llm_model.model_id

    def get_provider(self):
        model_id = self.get_model_name()
        return model_id.split('.')[0]

In [7]:
llama3_1_70b_deepeval = BedrockTextGenDeepEval(model=llama3_1_70b_langchain)

### Define simple RAG application
---

Let's define simple `chain` using `langchain` framework to get the source documents and LLM responses.

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = ('''
You are an expert, truthful assistant. You will be provided the task by human.
Use the given context only to respond to the request.

Here is the context: {context}
''')

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

qna_chain = create_stuff_documents_chain(llama3_70b_langchain, prompt_template)
rag_chain = create_retrieval_chain(chroma_retriver, qna_chain)

### Retrieval evaluation
---

In this section, we will focus on evaluating **retrieval** components.

- [**Contextual Precision**](https://docs.confident-ai.com/docs/metrics-contextual-precision): this metric measures whether the chunks in `retrieval_context` are relevant to the given `input`.
- [**Contextual Recall**](https://docs.confident-ai.com/docs/metrics-contextual-recall): this metric measures the quality of the retriever by evaluating the extent of which the `retrieval_context` aligns with the `expected_output`.
- [**Contextual Relevancy**](https://docs.confident-ai.com/docs/metrics-contextual-relevancy): this metric measures the overall relevance (or quality) of the information presented in your `retrieval_context` for a given `input`.

#### Define the metrics

In [9]:
from deepeval import evaluate
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric
)

contextual_precision_metric = ContextualPrecisionMetric(
    threshold=0.8,
    model=llama3_1_70b_deepeval,
    include_reason=True
)

contextual_recall_metric = ContextualRecallMetric(
    threshold=0.8,
    model=llama3_1_70b_deepeval,
    include_reason=True
)

contextual_relevancy_metric = ContextualRelevancyMetric(
    threshold=0.8,
    model=llama3_1_70b_deepeval,
    include_reason=True
)

Let's examine one example from evaluation dataset

In [10]:
import random

n = random.randrange(0, eval_df.shape[0])
print(n)

15


In [11]:
sample_question = eval_df.question[n]
expected_output = eval_df.ref_answer[n]
context = eval_df.source_chunk[n]
_rag_result = rag_chain.invoke({'input': sample_question})
llm_resp = _rag_result.get('answer', '').strip()
retrieval_context_list = [
    doc.page_content for doc in _rag_result.get('context')
]

In [12]:
from IPython.display import display, Markdown

display(Markdown(
    "<font color='blue'>'Question: {}".format(sample_question)
))
display(Markdown(
    "<font color='green'>'LLM response: {}".format(llm_resp)
))
display(Markdown(
    "<font color='brown'>'Expected response: {}".format(expected_output)
))

<font color='blue'>'Question: What is the goal for Alexa and Alexa-related devices in terms of their impact on customers' lives?

<font color='green'>'LLM response: The goal is for Alexa to be the world's most helpful and resourceful personal assistant, making people's lives "meaningfully easier and better".

<font color='brown'>'Expected response: The goal for Alexa and Alexa-related devices is to make people's lives meaningfully easier and better as the world's most helpful and resourceful personal assistant.

#### Construct LLMTestCase

In [13]:
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
    input=sample_question,
    actual_output=llm_resp,
    expected_output=expected_output,
    retrieval_context=retrieval_context_list,
    context=context
)

#### Get the metric
---
There are two methods to get the metric and reason

1. **One-by-one**: by using the metric's `measure` method on the `LLMTestCase`, you can get the metric evaluation one-by-one.
2. **Bulk**: you can pass multiple test cases and multiple evaluation metrics to DeepEval's `evaluate` method.

In [14]:
contextual_precision_metric.measure(test_case, _show_indicator=False)
display(Markdown(
    "<font color='blue'>'Contextual precision: {}".format(contextual_precision_metric.reason)
))

contextual_recall_metric.measure(test_case, _show_indicator=False)
display(Markdown(
    "<font color='green'>'Contextual recall: {}".format(contextual_recall_metric.reason)
))
contextual_relevancy_metric.measure(test_case, _show_indicator=False)
display(Markdown(
    "<font color='brown'>'Contextual relevancy: {}".format(contextual_relevancy_metric.reason)
))

Event loop is already running. Applying nest_asyncio patch to allow async execution...


<font color='blue'>'Contextual precision: The score is 1.00 because all relevant nodes in the retrieval context are ranked higher than irrelevant nodes, perfectly aligning with the desired outcome. The first node, ranked 1, directly addresses the goal for Alexa and Alexa-related devices, making it a highly relevant result.

Event loop is already running. Applying nest_asyncio patch to allow async execution...


<font color='green'>'Contextual recall: The score is 1.00 because the expected output perfectly aligns with the information provided in the 1st node in the retrieval context, accurately capturing the goal of Alexa and Alexa-related devices.

Event loop is already running. Applying nest_asyncio patch to allow async execution...


<font color='brown'>'Contextual relevancy: The score is 0.57 because although the retrieval context mentions the goal of Alexa being the world's most helpful and resourceful personal assistant, making people's lives easier and better, it also contains irrelevant information about other devices (Ring and Blink), failure, and lessons learned, which dilutes the relevance of the context to the input.

In [15]:
retrieval_report = evaluate(
    test_cases=[test_case], 
    metrics=[contextual_precision_metric, contextual_recall_metric, contextual_relevancy_metric],
    show_indicator=False,
    ignore_errors=True,
    print_results=True
)

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Metrics Summary

  - ✅ Contextual Precision (score: 1.0, threshold: 0.8, strict: False, evaluation model: meta.llama3-1-70b-instruct-v1:0, reason: The score is 1.00 because all relevant nodes in the retrieval context are ranked higher than irrelevant nodes, with the first node directly addressing the goal for Alexa and Alexa-related devices, making it a perfect ranking., error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.8, strict: False, evaluation model: meta.llama3-1-70b-instruct-v1:0, reason: The score is 1.00 because the expected output perfectly aligns with the information provided in the 1st node in the retrieval context, accurately capturing the goal of Alexa and Alexa-related devices., error: None)
  - ❌ Contextual Relevancy (score: 0.5714285714285714, threshold: 0.8, strict: False, evaluation model: meta.llama3-1-70b-instruct-v1:0, reason: The score is 0.57 because although th

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

### Generation evaluation
---

There are two main evaluation metrics for **generation** components, however, you can add other metrics to fit your use cases.

- [**Answer Relevancy**](https://docs.confident-ai.com/docs/metrics-answer-relevancy): this metric measures how relevant the `actual_output` of your LLM application is compared to the provided `input`.
- [**Faithfulness**](https://docs.confident-ai.com/docs/metrics-faithfulness): this metric measures the generator by evaluating whether the `actual_output` factually <u>aligns with the contents of your `retrieval_context`</u>.
- [**Bias**](https://docs.confident-ai.com/docs/metrics-bias): this metric measures whether your LLM outputs contain gender, racial, or political bias
- [**Toxicity**](https://docs.confident-ai.com/docs/metrics-toxicity): this metric measures whether your LLM outputs contain any toxicity.

#### Define the metrics

In [16]:
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    BiasMetric,
    ToxicityMetric
)

answer_relevancy_metric = AnswerRelevancyMetric(
    threshold=0.8,
    model=llama3_1_70b_deepeval,
    include_reason=True
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.8,
    model=llama3_1_70b_deepeval,
    include_reason=True
)

bias_metric = BiasMetric(
    threshold=0.8,
    model=llama3_1_70b_deepeval,
    include_reason=True
)


toxicity_metric = ToxicityMetric(
    threshold=0.8,
    model=llama3_1_70b_deepeval,
    include_reason=True
)

#### Construct LLMTestCase
---
Let's reuse the same question and answer for this.

In [17]:
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
    input=sample_question,
    actual_output=llm_resp,
    expected_output=expected_output,
    retrieval_context=retrieval_context_list,
    context=context
)

#### Get the metric
---
There are two methods to get the metric and reason. This is the same as previously in **Retrieval** section.

1. **One-by-one**: by using the metric's `measure` method on the `LLMTestCase`, you can get the metric evaluation one-by-one.
2. **Bulk**: you can pass multiple test cases and multiple evaluation metrics to DeepEval's `evaluate` method.

In [18]:
from IPython.display import display, Markdown

answer_relevancy_metric.measure(test_case, _show_indicator=False)
display(Markdown(
    "<font color='blue'>'Answer relevancy: {}".format(answer_relevancy_metric.reason)
))
faithfulness_metric.measure(test_case, _show_indicator=False)
display(Markdown(
    "<font color='green'>'Faithfulness: {}".format(faithfulness_metric.reason)
))
bias_metric.measure(test_case, _show_indicator=False)
display(Markdown(
    "<font color='brown'>'Bias: {}".format(bias_metric.reason)
))
toxicity_metric.measure(test_case, _show_indicator=False)
display(Markdown(
    "<font color='brown'>'Toxicity: {}".format(toxicity_metric.reason)
))

Event loop is already running. Applying nest_asyncio patch to allow async execution...


<font color='blue'>'Answer relevancy: The score is 1.00 because the actual output perfectly addresses the question, providing a clear and relevant response that aligns with the input, with no irrelevant statements detected!

Event loop is already running. Applying nest_asyncio patch to allow async execution...


<font color='green'>'Faithfulness: The score is 1.00 because there are no contradictions found, indicating a perfect alignment between the actual output and the retrieval context!

Event loop is already running. Applying nest_asyncio patch to allow async execution...


<font color='brown'>'Bias: The score is 0.00 because the actual output appears to be unbiased, as there are no reasons listed to suggest otherwise, indicating a perfectly neutral result.

Event loop is already running. Applying nest_asyncio patch to allow async execution...


<font color='brown'>'Toxicity: The score is 0.00 because the actual output is completely non-toxic and respectful, with no harmful or offensive content detected.

In [19]:
generator_report = evaluate(
    test_cases=[test_case], 
    metrics=[
        answer_relevancy_metric, faithfulness_metric, bias_metric, toxicity_metric
    ],
    show_indicator=False,
    ignore_errors=True,
    print_results=True
)

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.8, strict: False, evaluation model: meta.llama3-1-70b-instruct-v1:0, reason: The score is 1.00 because the output perfectly addressed the question about Alexa's goal, providing a clear and relevant response without any irrelevant information., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.8, strict: False, evaluation model: meta.llama3-1-70b-instruct-v1:0, reason: The score is 1.00 because there are no contradictions, indicating a perfect alignment between the actual output and the retrieval context!, error: None)
  - ✅ Bias (score: 0.0, threshold: 0.8, strict: False, evaluation model: meta.llama3-1-70b-instruct-v1:0, reason: The score is 0.00 because the actual output appears to be unbiased, as there are no reasons listed to suggest otherwise., error: None)
  - ✅ Toxicity (score: 0.0, threshold: 0.8, strict: False, evalu

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

## Summary
---

`DeepEval` is an open-source Python library specifically designed for evaluating Retrieval-Augmented Generation (RAG) applications. It offers a comprehensive set of tools and metrics to assess various aspects of RAG performance, including retrieval accuracy, generation quality, faithfulness to source material, and overall coherence. By decoupling the metrics into **retrieval** and **generation** components, developers can assess, and pinpoint the components need to be improved.

`DeepEval` enables automated testing and benchmarking, allowing for efficient comparison of different RAG models or configurations. The library supports customizable metrics, scalable evaluations, and provides detailed insights into each component of the RAG system. This level of comprehensive analysis helps in identifying areas for improvement, optimizing performance, and ultimately enhancing the user experience. By integrating DeepEval into the development workflow, teams can make data-driven decisions, save time and resources, and ensure the continuous improvement of their RAG applications.

However, its reliance on **Large Language Model (LLM)** for assessment introduces potential drawbacks, including bias, increased costs, dependency issues, and consistency concerns. While `DeepEval` provides detailed insights and facilitates data-driven optimization of RAG systems, <u>users should be aware of these limitations and consider supplementary evaluation methods to ensure a well-rounded assessment of their RAG applications</u>.